# Climate correction

The present-day temperature perturbation, $\Delta T(z, t=0)$, in a semi-infinite solid with an instantaneous change of surface temperature $\Delta T$ at time $t$ before the present is:

$$ \Delta T(z,t=0) = \Delta T  \; \mathrm{erfc} \left(\frac{z}{2 \sqrt{\kappa t}} \right)$$

The effect of more than one event, $k_1, k_2, \ldots, k_n$, is found by summation - i.e. if $T(z=0,t) = T_k$ for $t_{k-1} < t < t_k$:

$$ \Delta T(z,t=0) = \sum_{k=1}^{n} T_k \left[ \mathrm{erfc}\left(\frac{z}{2 \sqrt{\kappa t_k}} \right) - \mathrm{erfc}\left(\frac{z}{2 \sqrt{\kappa t_{k-1}}} \right) \right] $$

We use this formula if the temperature has remained constant over a period of time in the past.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

import csv
import os
from scipy.interpolate import UnivariateSpline
from scipy import odr
from scipy.special import erfc

In [ ]:
def load_borehole(filename):
    z, T, sigma_T   = np.loadtxt(filename, delimiter=',', unpack=True, skiprows=1, usecols=(0,1,2), dtype=str)
    z_k, k, sigma_k = np.loadtxt(filename, delimiter=',', unpack=True, skiprows=1, usecols=(3,4,5), dtype=str)
    
    def trim_columns(*args):
        trim_args = [None]*len(args)
        for a, arg in enumerate(args):
            trim_args[a] = arg[arg != ''].astype(float)
        return trim_args
            
    # trim columns
    z, T, sigma_T, z_k, k, sigma_k = trim_columns(z, T, sigma_T, z_k, k, sigma_k)

    # trim z_k and k values that are above our temperature range
    mask = z_k <= z.max()
    z_k = z_k[mask]
    k = k[mask]
    sigma_k = sigma_k[mask]
    return z, T, sigma_T, z_k, k, sigma_k


def thermal_resistance(z, k):
    delta_z = np.diff(np.hstack([[0.0], z_k]))
    return np.cumsum(delta_z/k)

def least_squares(R, T):
    A = np.vstack([R, np.ones(len(R))]).T
    m, c = np.linalg.lstsq(A, T)[0]
    return m, c

def linear_func(p, x):
    m, c = p
    return m*x + c

def climate_correction_period(kappa, t0, t1, Tk, z):
    return Tk*(erfc(z/(2.0*np.sqrt(kappa*t1))) - erfc(z/(2.0*np.sqrt(kappa*t0))))


def odr_model(func, R, T, sigma_R, sigma_T):
    model = odr.Model(func)
    data = odr.RealData(R, T, sx=sigma_R, sy=sigma_T)
    
    A = np.vstack([R, np.ones(len(R))]).T
    m, c = np.linalg.lstsq(A, T_k)[0]
    x0 = [m, c]
    
    reg = odr.ODR(data, model, beta0=x0)
    out = reg.run()
    return out


def MC_simulation(nsim, t0, t1, Tk, z, k, cp=800.0, rho=2700.0):
    delT = np.zeros((nsim, z.size))
    k_mean = k.mean()
    for i in range(0, nsim):
        t0_random = np.clip(t0, 1e-6, 1e99) #np.random.normal(t0, sigma_t0)
        t1_random = np.clip(t1, 1e-6, 1e99) #np.random.normal(t1, sigma_t1)
        Tk_random = np.random.normal(Tk[0], Tk[1])
        k_random = np.clip(np.random.normal(k_mean, 0.25*k_mean), 0.001, 1e99)
        kappa = k_random/(cp*rho)
        delT[i] = climate_correction_period(kappa, t0_random, t1_random, Tk_random, z)
    return np.mean(delT, axis=0), np.std(delT, axis=0)

def y2s(t):
    "years to seconds"
    return t*3.15567*1e7
def s2y(t):
    return t/3.15567/1e7

Import palaeoclimate data Thomas Farrell's compilation

In [ ]:
palaeoclimate = np.loadtxt('../data/surface_temperature_history.csv', delimiter=',', skiprows=1)
t0 = y2s(palaeoclimate[:,1]*1e3)
t1 = y2s(palaeoclimate[:,0]*1e3)
Tc       = palaeoclimate[:,2]
sigma_Tc = palaeoclimate[:,3]

Import warming data for the past 100+ years from NASA

In [ ]:
year, T0_40N = np.loadtxt('../data/ZonAnn.Ts+dSST.csv', delimiter=',', skiprows=1, unpack=True, usecols=(0,4))
year = y2s(1974 - year) # boreholes measured in 1974
T0_40N = T0_40N - T0_40N[-1]

In [ ]:
t_av = 10

t0b = np.zeros(year.size//10+1)
T0_40N2 = np.zeros(year.size//10+1)

index = 0
for i in range(0, year.size, t_av):
    t0b[index]  = np.mean(year[i:i+t_av])
    T0_40N2[index] = np.mean(T0_40N[i:i+t_av])
    index += 1
    
    
t1b = t0b[0:-1] + np.diff(t0b)
t1b = np.hstack([t1b, [0.0]])

Plot surface temperature history

In [ ]:
temp_ago = np.column_stack([Tc, Tc]).ravel()
time_ago = np.column_stack([t1, t0]).ravel()
time_ago = s2y(time_ago)


fig = plt.figure(figsize=(5,3.5))
ax1 = fig.add_subplot(111, xlim=(0,150))
ax1.set_ylabel(r'$\Delta T_0$ ($^{\circ}$C)')
ax1.set_xlabel('time before present (ka)')

ax1.plot(time_ago/1e3, temp_ago, 'k-', label=r'$\Delta T_c$')
ax1.legend(loc='lower right', frameon=False, borderpad=1)

# Bullard plots

A Bullard plot graphs thermal resistance (m$^2$K W$^{-1}$) against temperature. The gradient of this line is heat flow, and the uncertainty determined from the error of linear regression.

Thermal resistance is calculated by:

$$R = \sum_{i=0}^{n} \left( \frac{\Delta z_i}{k_i} \right)$$

In [ ]:
import csv
import os
from difflib import get_close_matches
filedir = '../data/boreholes/'

csvfiles = []
for f in os.listdir(filedir):
    if f.endswith('.csv'):
        csvfiles.append(os.path.join(filedir, f))

In [ ]:
# constants
cp = 800.0
rho = 2700.0
nsim = 10000

In [ ]:
# create summary data file
outputFile = filedir+'borehole_summary.dat'

header = ['Borehole', 'Qs', 'dQs', 'climate Qs', 'climate dQs']
outfile = open(outputFile, 'w')
csvwriter = csv.writer(outfile, delimiter=',')
csvwriter.writerow(header)


# create storage arrays
q_all  = np.zeros(nsim)
qc_all = np.zeros(nsim)
c_all  = np.zeros(nsim)
cc_all = np.zeros(nsim)


for filename in csvfiles:
    borename = os.path.basename(filename)
    borename = os.path.splitext(borename)[0]
    borename = borename.replace('_', ' ')
    
    # load file
    z, T, sigma_T, z_k, k, sigma_k = load_borehole(filename)
    
    # Spline interpolation to thermal conductivity data
    if len(z) <= 3:
        T_k = T
        sigma_T_k = sigma_T
    else:
        spl = UnivariateSpline(z, T)
        T_k = spl(z_k)
        spl = UnivariateSpline(z, sigma_T)
        sigma_T_k = spl(z_k)
        
    T_all = np.zeros((nsim, T_k.size))

    # SENSITIVITY ANALYSIS
    # Perturb input parameters
    for i in range(0, nsim):
        T_random = np.random.normal(T_k, sigma_T_k)
        k_random = np.clip(np.random.normal(k, sigma_k), 0.001, 1e99) # conductivity
        a_random = k_random/(cp*rho) # diffusivity

        # Calculate thermal resistance
        R = thermal_resistance(z_k, k_random)

        # Compute heat flow
        q, c = least_squares(R, T_random)

        # PALAEOCLIMATE
        # climate-correction over 100k timespan
        for j in range(0, t0.size):
            t0_random = np.clip( np.random.normal(t0[j], y2s(1e3)) , 1e-6, 1e99)
            t1_random = np.clip( np.random.normal(t1[j], y2s(1e3)) , 1e-6, 1e99)
            Tc_random = np.random.normal(Tc[j], sigma_Tc[j])
            T_random += climate_correction_period(a_random, t0_random, t1_random, Tc_random, z_k)

        # climate-correction from NASA
        for j in range(0, t0b.size):
            t0_random = np.clip(t0b[j], 1e-6, 1e99)
            t1_random = np.clip(t1b[j], 1e-6, 1e99)
            Tc_random = np.random.normal(T0_40N2[j], 0.1)
            T_random += climate_correction_period(a_random, t0_random, t1_random, Tc_random, z_k)

        # calculate corrected heat flow
        qc, cc = least_squares(R, T_random)
        
        # store variables
        q_all[i] = q
        c_all[i] = c
        qc_all[i] = qc
        cc_all[i] = cc
        
        # temperature correction for Bullard plot
        T_all[i] = T_random
        
        
    Qs   = np.mean(q_all) *1e3
    Qs_c = np.mean(qc_all) *1e3
    sigma_Qs   = np.std(q_all) *1e3
    sigma_Qs_c = np.std(qc_all) *1e3


    # create arrays for plotting
    R = thermal_resistance(z_k, k)
    sigma_z = np.zeros_like(z_k)
    for i in range(0, len(z_k)):
        sigma_z[i] = np.abs(z_k[i] - z).min()

    delta_z = np.diff(np.hstack([[0.0], z_k]))
    invk = delta_z/k
    dinvk = np.sqrt((sigma_k/k)**2 + (sigma_z/delta_z)**2) * invk
    sigma_R = np.sqrt(np.cumsum(dinvk**2))
    
    Tc_mean = np.mean(T_all, axis=0)
    Tc_std  = np.sqrt(np.mean(np.abs(T_all - Tc_mean)**2, axis=0))
    
    
    # Save bullard plot
    fig = plt.figure(figsize=(5,3.5))
    ax1 = fig.add_subplot(111, xlabel=r'thermal resistance (m$^2$K W$^{-1}$)', ylabel=r'temperature ($^{\circ}$C)')
    ax1.patch.set_alpha(1.0)
    ax1.errorbar(R, T_k, np.ones_like(T_k) * sigma_T_k, sigma_R, fmt='o', capsize=3, markeredgecolor='k', markeredgewidth=0.1, zorder=3, label='uncorrected')
    ax1.errorbar(R, Tc_mean, Tc_std, sigma_R, fmt='o', capsize=3, markeredgecolor='k', markeredgewidth=0.1, zorder=3, label='climate-corrected')
    ax1.plot(R, linear_func((np.mean(q_all), np.median(c_all)), R), zorder=4, color='C0')
    ax1.plot(R, linear_func((np.mean(qc_all), np.median(cc_all)), R), zorder=5, color='C1')
    ax1.legend(loc='upper left')
    textstr1 = "$q_s = %1.1f \, \mathrm{mW \, m^{-2}}$\n$\sigma_{q_{s}} = %1.1f \, \mathrm{mW \, m^{-2}}$" % (Qs, sigma_Qs)
    textstr2 = "$q_s = %1.1f \, \mathrm{mW \, m^{-2}}$\n$\sigma_{q_{s}} = %1.1f \, \mathrm{mW \, m^{-2}}$" % (Qs_c, sigma_Qs_c)
    fig.text(0.5, 0.2, textstr1, color="C0")
    fig.text(0.2, 0.5, textstr2, color="C1")
    fig.savefig(filedir+borename+'.pdf', bbox_inches='tight', transparent=True)
    plt.close(fig)


    print("{:25}| q = {:.2f} +- {:.2f},  qc = {:.2f} +- {:.2f}".format(borename,\
                                                                       Qs, sigma_Qs,\
                                                                       Qs_c, sigma_Qs_c))

    csvwriter.writerow([borename, Qs, sigma_Qs, Qs_c, sigma_Qs_c])
    
outfile.close()

## Palaeoclimate correction

In [ ]:
def MC_simulation_iter(nsim, t0, t1, Tk, z, k, cp=800.0, rho=2700.0):
    delT = np.zeros((nsim, z.size))
    k_mean = k.mean()
    for i in range(0, nsim):
        t0_random = np.clip( np.random.normal(t0, y2s(1e3)) , 1e-6, 1e99)
        t1_random = np.clip( np.random.normal(t1, y2s(1e3)) , 1e-6, 1e99)
        Tk_random = np.random.normal(Tk[0], Tk[1])
        k_random = np.clip(np.random.normal(k_mean, 0.2*k_mean), 0.001, 1e99)
        kappa = k_random/(cp*rho)
        delT[i] = climate_correction_period(kappa, t0_random, t1_random, Tk_random, z)
    return delT

In [ ]:
filename = csvfiles[-1]

borename = os.path.basename(filename)
borename = os.path.splitext(borename)[0]
borename = borename.replace('_', ' ')

# generate some random data
z_k = np.linspace(1, 5e3, 10000)
k = np.ones_like(z_k) * 3.2

# Correct for climate effects
nsim = 10000
delTN = np.zeros((nsim, k.size))

for i in range(0, t0.size):
    delTN += MC_simulation_iter(nsim, t0[i], t1[i], (Tc[i], 1.0), z_k, k)
for i in range(0, t0b.size):
    delTN += MC_simulation_iter(nsim, t0b[i], t1b[i], (T0_40N2[i], 0.1), z_k, k)


In [ ]:
z_kN = np.tile(z_k, nsim)

# create a heat map of palaeoclimate realisations
heatmap, xedges, yedges = np.histogram2d(z_kN, delTN.ravel(), bins=100, normed=True)


In [ ]:
fig = plt.figure(figsize=(4,4))
ax1 = fig.add_subplot(111, xlabel=r'$\Delta T$', ylabel='depth (km)')
im1 = ax1.contourf(heatmap, origin='upper', extent=[delTN.min(), delTN.max(), z_k.min(), z_k.max()],
                   cmap='inferno', aspect=0.004)

rect = mpatches.Rectangle((-1,4000), 7, 1000, fill=False, edgecolor='w', linestyle='--')
ax1.add_patch(rect)

ylabels = []
for label in ax1.get_yticks():
    ylabels.append(int(label*1e-3))
ax1.set_yticklabels(ylabels[::-1])
fig.colorbar(im1, label='probability density')

ax2 = fig.add_axes([0.15, 0.38, 0.28, 0.33], ylim=[4000, 5000], xlim=[-1,6])
im2 = ax2.contourf(np.flipud(heatmap), extent=[delTN.min(), delTN.max(), z_k.min(), z_k.max()],
                   cmap='inferno', aspect=0.004)
ax2.spines['bottom'].set_color('white')
ax2.spines['top'].set_color('white')
ax2.spines['left'].set_color('white')
ax2.spines['right'].set_color('white')
ax2.xaxis.label.set_color('white')
ax2.tick_params(axis='x', colors='white')
ax2.tick_params(axis='y', colors='white')
ax2.set_yticks([])
ax2.set_xticks([0, 2, 4, 6])
# ylabels = []
# for label in ax2.get_yticks():
#     ylabels.append(int(label*1e-3))
# ax2.set_yticklabels(ylabels[::-1])


fig.savefig("palaeoclimate_correction_heatmap.pdf", dpi=300, bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(4,4))
ax1 = fig.add_subplot(111, xlabel=r'$\Delta T$', ylabel='depth (km)')
im1 = ax1.contourf(np.flipud(heatmap), extent=[delTN.min(), delTN.max(), z_k.min(), z_k.max()],
                   cmap='inferno', aspect=0.004)
ylabels = []
for label in ax1.get_yticks():
    ylabels.append(int(label*1e-3))
ax1.set_yticklabels(ylabels[::-1])
fig.colorbar(im1, label='probability density')
fig.savefig("palaeoclimate_correction_heatmap.pdf", dpi=300, bbox_inches='tight')